# Parameter exploration of a brain network

This notebook shows how to scan the parameter space of a more complex model, namely a brain network consisting of coupled `aln` populations. 

The evaluation of the simulation done in multiple steps: 
- Step 1 checks if there is any rate activity at all
- Step 2 checks if there is any BOLD activity
- Step 3 is the full simulation

After the full simulation, the funciotnal connectivity (FC) of the BOLD signal is computed and compared to the empirical FC dataset. The Pearson correlation of the FC matrices is computed and the average is taken. This is the result that we tell pypet to save. We could also save the rate output of the model or the BOLD timeseries. 

In [1]:
# change into root directory to the project
import os
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

%load_ext autoreload
%autoreload 2

In [3]:
import pypet
import numpy as np

from neurolib.models.aln import ALNModel
from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.exploration import BoxSearch
import neurolib.utils.functions as func

from neurolib.utils.loadData import Dataset
ds = Dataset("hcp")

In [4]:
aln = ALNModel(Cmat = ds.Cmat, Dmat = ds.Dmat, bold=True) # simulates the whole-brain model in 10s chunks by default if bold == True
# Resting state fits
aln.params['mue_ext_mean'] = 1.57
aln.params['mui_ext_mean'] = 1.6
aln.params['sigma_ou'] = 0.09
aln.params['b'] = 5.0
aln.params['signalV'] = 2
aln.params['dt'] = 0.2
aln.params['duration'] = 0.2 * 60 * 1000 #ms
# testing: aln.params['duration'] = 0.2 * 60 * 1000 #ms
# real: aln.params['duration'] = 1.0 * 60 * 1000 #ms

INFO:root:Model aln created


In [27]:
def evaluateSimulation(traj):
    # get the model from the trajectory using `search.getModelFromTraj(traj)`
    model = search.getModelFromTraj(traj)
    defaultDuration = model.params['duration']
    invalid_result = {"fc" : [0]* len(ds.BOLDs)}
    
    # -------- stage wise simulation --------
    
    # Stage 1 : simulate for a few seconds to see if there is any activity
    # ---------------------------------------
    model.params['dt'] = 0.1
    model.params['duration'] = 3*1000.
    model.run()
    
    # check if stage 1 was successful
    if np.max(aln.rates_exc[:, aln.t > 500]) > 300 or np.max(aln.rates_exc[:, aln.t > 500]) < 10:
        search.saveOutputsToPypet(invalid_result, traj)
        return invalid_result, {}
    
    # Stage 2: simulate BOLD for a few seconds to see if it moves
    # ---------------------------------------
    model.params['dt'] = 0.2
    model.params['duration'] = 20*1000.
    model.run(bold = True)
    
    if np.std(aln.BOLD.BOLD[:, 5:10]) < 0.001:
        search.saveOutputsToPypet(invalid_result, traj)
        return invalid_result, {}
    
    # Stage 3: full and final simulation
    # ---------------------------------------
    model.params['dt'] = 0.2
    model.params['duration'] = defaultDuration
    model.run(bold = True)
    
    # -------- evaluation here --------
    
    scores = []
    for i, fc in enumerate(ds.FCs):#range(len(ds.FCs)):
        fc_score = func.matrix_correlation(func.fc(model.BOLD.BOLD[:, 5:]), fc)
        scores.append(fc_score)
    
    meanScore = np.mean(scores)
    result_dict = {"fc" : meanScore}
    
    # Save the results to pypet. 
    # Remember: This has to be dictionary!
    # This could be the activity or some abstract result like
    # the average FC correlation here
    search.saveOutputsToPypet(result_dict, traj)

In [ ]:
parameters = ParameterSpace({"mue_ext_mean": np.linspace(0, 3, 2), "mui_ext_mean": np.linspace(0, 3, 2)})
# info: chose np.linspace(0, 3, 21) or more, values here are low for testing
search = BoxSearch(evalFunction = evaluateSimulation, model=aln, exploreParameters=parameters)

In [ ]:
search.run()

In [ ]:
search.loadResults()
print("Number of results: {}".format(len(search.results)))

In [37]:
for i in search.dfResults.index:
    search.dfResults.loc[i, 'fc'] = np.mean(search.results[i]['fc'])
search.dfResults

,mue_ext_mean,mui_ext_mean,fc
0,0.0,0.0,0.000000
1,0.0,3.0,0.000000
2,3.0,0.0,0.042386
3,3.0,3.0,-0.182764
